In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#importing the datasets
customers_df = pd.read_csv('customers_dataset.csv')
customers_df.head()
geo_df = pd.read_csv('geolocation_dataset.csv')
geo_df.head()
order_payments_df = pd.read_csv('order_payments_dataset.csv')
order_payments_df.head()
products_df = pd.read_csv('products_dataset.csv')
products_df.head()
sellers_df = pd.read_csv('sellers_dataset.csv')
sellers_df.head()
orders_df = pd.read_csv('orders_dataset.csv') 
orders_df.head()
order_items_df = pd.read_csv('order_items_dataset.csv')
order_items_df.head()
order_reviews_df = pd.read_csv('order_reviews_dataset.csv')
order_reviews_df.head()
product_category_name_translation_df = pd.read_csv('product_category_name_translation.csv')

# Exploring the datasets
customers_df.info()
geo_df.info()
order_payments_df.info()
products_df.info()
sellers_df.info()
orders_df.info()
order_items_df.info()
order_reviews_df.info()
product_category_name_translation_df.info()
#null values were not found in any of the datasets

# Recency, Frequency and Monetary Value
rfm_data = orders_df.merge(order_payments_df, on="order_id")
#converting date strings to datetime objects
orders_df


FileNotFoundError: [Errno 2] No such file or directory: 'null/Users/oc25ale1917'